# Fitting crescita COVID 19

In [1]:
#Importazione librerie operative
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import curve_fit
from ipywidgets import interact

In [2]:
#Lettura dati sperimentali via web
df = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')

### Impostazione funzione di fitting - curva "logistica"
  
### $$ N(N_{tot},N_0,r,t) = \frac{N_{tot}N_0}{N_0+(N_{tot}-N_0)e^{-rt}}$$


In [3]:
def sigma(x,a,b,c):
    y=(a*b)/(b+(a-b)*np.exp(-c*x))
    return y

In [11]:
Nexp = df['totale_casi']
a = df.count()
t = 1+np.arange(a['data'])
t1 = np.arange(70)
t2 = t1-1

In [12]:
df_sel=df.drop(['data','stato','note_it', 'note_en'],axis=1)
@interact(parametro=df_sel.columns)
def sel(parametro):
    fg=go.Figure()
    return fg.add_trace(go.Scatter(x=df.data ,y=df[parametro], mode='markers', marker_color = 'green'))

interactive(children=(Dropdown(description='parametro', options=('ricoverati_con_sintomi', 'terapia_intensiva'…

##### Fit dati sperimentali

In [13]:
popt, pcov = curve_fit(sigma, t, Nexp, maxfev=5000, method='trf', bounds=(0, [200000., 1500., 0.5]))
fit = sigma(t, *popt)
k, N0, r = popt
der = sigma(t1,*popt) - sigma(t2,*popt)
print('Ntotale=' ,int(popt[0])  ,'N0=' ,int(popt[1])  ,'r=' ,"%0.2f" % popt[2])

Ntotale= 192603 N0= 1499 r= 0.14


### Plot risultati ottenuti

In [14]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=t, y=Nexp, mode='markers',name='n°casi totali', marker=dict(symbol=[200],color='red', size=8)))
fig.add_trace(go.Scatter(x=t1, y=sigma(t1,*popt), mode='lines',name='Previsione',marker_color='green'))
fig.update_layout(title='Crescita casi COVID 19', \
                  xaxis_title='giorni da 24 feb',yaxis_title='n° totale casi')

In [15]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=t, y=df['nuovi_positivi'], mode='markers',name='incremento casi/giorno', \
                         marker=dict(symbol=[200],color='red', size=10)))
fig.add_trace(go.Scatter(x=t1, y=der, mode='lines',name='Previsione',marker_color='blue'))
fig.update_layout(title='Incremento giornaliero positivi COVID 19', \
                  xaxis_title='giorni da 24 feb',yaxis_title='n° casi giornalieri')

In [9]:
tab=df.drop(['stato','note_it', 'note_en'],axis=1)
tab

,data,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,101,26,127,94,221,0,221,1,7,229,4324,NaN
1,2020-02-25T18:00:00,114,35,150,162,311,90,93,1,10,322,8623,NaN
2,2020-02-26T18:00:00,128,36,164,221,385,74,78,3,12,400,9587,NaN
3,2020-02-27T18:00:00,248,56,304,284,588,203,250,45,17,650,12014,NaN
4,2020-02-28T18:00:00,345,64,409,412,821,233,238,46,21,888,15695,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2020-04-25T17:00:00,21533,2102,23635,82212,105847,-680,2357,63120,26384,195351,1707743,1186526.0
62,2020-04-26T17:00:00,21372,2009,23381,82722,106103,256,2324,64928,26644,197675,1757659,1210639.0
63,2020-04-27T17:00:00,20353,1956,22309,83504,105813,-290,1739,66624,26977,199414,1789662,1237317.0
64,2020-04-28T17:00:00,19723,1863,21586,83619,105205,-608,2091,68941,27359,201505,1846934,1274871.0
